In [1]:
from tensorflow.keras.models import load_model
import numpy as np
from skimage import transform, io, img_as_float, exposure, morphology, img_as_ubyte
import matplotlib.pyplot as plt
import os
from PIL import Image
from shutil import move
import cv2 

In [2]:
BASE_PATH = os.path.dirname(os.path.realpath(__name__))
BASE_PATH = '/Users/alvaromoure/Desarrollo/UPM/TFM/Alvaro-TFM'

In [3]:
# Load model

# model_name = '/home/julian/Documents/PythonExperiments/COVIDNet/segmenter/trained_model.hdf5'
model_name = os.path.join(BASE_PATH,'segmenter','trained_model.hdf5')
UNet = load_model(model_name)

2022-05-08 11:47:06.787313: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def loadDataGeneral(path, im_shape):
    """Function for loading arbitrary data in standard formats"""
    X = []   
    image = Image.open(path).convert('RGB')
    image = cv2.imread(path)
    #image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = np.array(image)
    img = img_as_float(image)
    img = transform.resize(img, im_shape)
    img = exposure.equalize_hist(img)
    img = np.expand_dims(img, -1)
    X.append(img)
    X = np.array(X)
    X -= X.mean()
    X /= X.std()
    return X, img

In [6]:
def remove_small_regions(img, size):
    """Morphologically removes small (less than size) connected regions of 0s or 1s."""
    img = morphology.remove_small_objects(img, size)
    img = morphology.remove_small_holes(img, size)
    return img

In [7]:
im_shape = (256, 256)
#path = '/home/julian/Documents/PythonExperiments/COVIDNet/Datos/DatosExperimentos_3/OrgImages/'
#path = os.path.join(BASE_PATH,'data','OrgImages\\')
path = '/Volumes/GoogleDrive/Mi unidad/UPM/TFM_UPM/datasets/hackathon/TrainSetGood/'
#path2 = '/home/julian/Documents/PythonExperiments/COVIDNet/Datos/DatosExperimentos_3/RemoveSegmented/'
path2 = os.path.join(BASE_PATH,'data','RemoveSegmented')
#path3 = '/home/julian/Documents/PythonExperiments/COVIDNet/Datos/DatosExperimentos_3/BrokenFiles/'
path3 = os.path.join(BASE_PATH,'data','BrokenFiles')

path4 = os.path.join(BASE_PATH,'data','OrgImagesSegmented')

if not os.path.exists(path): 
  os.makedirs(path)

if not os.path.exists(path2): 
  os.makedirs(path2)

if not os.path.exists(path3): 
  os.makedirs(path3)

if not os.path.exists(path4): 
  os.makedirs(path4)

arr = os.listdir(path + '.')
arr.sort()
for j,file in enumerate(arr):
    try:
        image_name = path + file
        X,img_org = loadDataGeneral(image_name, im_shape)
        img = exposure.rescale_intensity(np.squeeze(X), out_range=(0,1))
        img = np.sum(img,axis=2)[None,:,:,None]
        pred = UNet.predict(img)
        #-------------------------
        pr = pred > 0.5
        # Remove regions smaller than 2% of the image
        #pr = remove_small_regions(pr, 0.02 * np.prod(im_shape))
        pr2 = np.squeeze(pr)
        seg_img = np.squeeze(img_org)*pr2[:,:,None]#np.dstack((pr2, pr2, pr2))
        maximo = np.max(seg_img)
        minimo = np.min(seg_img)
        if (maximo==0) and (minimo == 0):
            #print(file)
            move(image_name,path2 + file)
        else:
            seg_img /= maximo
            #seg_img = (seg_img - minimo)/(maximo-minimo)
            #io.imshow(seg_img)
            save_path = os.path.join(path4,file)
            io.imsave(save_path,img_as_ubyte(seg_img))
    except Exception as e:
      print(e)
      move(image_name,os.path.join(path3,file))
      #break

/Users/alvaromoure/Desarrollo/UPM/LSMA/venv/lib/python3.8/site-packages/skimage/_shared/utils.py:394: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel, or set channel_axis.
  return func(*args, **kwargs)


KeyboardInterrupt: 